In [1]:
import pandas as pd
import numpy as np
import re
import spacy
import tensorflow as tf
from langdetect import detect
from transformers import BertTokenizer
from transformers import TFBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

2023-05-15 17:07:21.429181: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 17:07:21.709445: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 17:07:22.910200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-15 17:07:25.342920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 1

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
df = pd.read_csv('Hindu_articles.csv')

In [4]:
df.dropna(inplace=True)

In [5]:
df.head()

,Title,Article
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2316 entries, 0 to 2323
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    2316 non-null   object
 1   Article  2316 non-null   object
dtypes: object(2)
memory usage: 54.3+ KB


In [7]:
df.shape

(2316, 2)

In [8]:
articles = df['Article']

In [9]:
articles

0       Domestic passenger vehicle wholesales grew 26....
1       Automobili Lamborghini has unveiled Urus S, th...
2       International trade in the domestic currency w...
3       G-20 countries discussed the debt distress of ...
4       India’s goods exports declined for the second ...
                              ...                        
2319    B. Suresh’s ward Kings Walk (P. Sai Kumar up) ...
2320     Romualdo, who maintains form, may repeat in t...
2321    Maharashtra Assembly Speaker Rahul Narwekar sa...
2322    The Bharatiya Janata Party (BJP) in Uttar Prad...
2323    The Biju Janata Dal on May 13 registered a lan...
Name: Article, Length: 2316, dtype: object

In [10]:
text = articles[10]
text

'The Indian Railways turns 170 on April 16. Forever India’s favourite carrier, the railways are as much at a crossroads today as the rest of India — reconciling the trials and tribulations of the past with a future reckoning.\nPerhaps the most striking symbols of Indian Railways in the current zeitgeist are the talismanic Vande Bharat Express trains. Much of this modernisation contributes to passenger comfort, but sometimes it also leads to a spectacularly short-sighted loss of beauty and cultural heritage.\nIn cities across India, colonial-era station buildings, many of which are a syncretic mix of local culture and European styles — such as Kacheguda in Hyderabad and Egmore in Chennai — are set for modernisation that will almost certainly produce drab, mall-like monoliths.\n\nKacheguda station in Hyderabad is one among the\xa01,275 railway stations slated for redevelopment under the Amrit Bharat scheme.\n| Photo Credit:\nKartik T.\n\n\nEgmore station in Chennai will be redeveloped at

In [11]:
doc = nlp(text)
for sentence in doc.sents:
    print(sentence.text)

The Indian Railways turns 170 on April 16.
Forever India’s favourite carrier, the railways are as much at a crossroads today as the rest of India — reconciling the trials and tribulations of the past with a future reckoning.

Perhaps the most striking symbols of Indian Railways in the current zeitgeist are the talismanic Vande Bharat Express trains.
Much of this modernisation contributes to passenger comfort, but sometimes it also leads to a spectacularly short-sighted loss of beauty and cultural heritage.

In cities across India, colonial-era station buildings, many of which are a syncretic mix of local culture and European styles — such as Kacheguda in Hyderabad and Egmore in Chennai — are set for modernisation that will almost certainly produce drab, mall-like monoliths.


Kacheguda station in Hyderabad is one among the 1,275 railway stations slated for redevelopment under the Amrit Bharat scheme.

| Photo Credit:
Kartik T.


Egmore station in Chennai will be redeveloped at a cost o

In [12]:


df['Article'] = df['Article'].astype(str)

for index, row in df.iterrows():
    article = row['Article']
    pattern1 = re.compile(r'\n\nCOMMents\n\n SHARE\n\n.*(?:\n.*)*')
    article = re.sub(pattern1, '', article)
    pattern2 = re.compile(r'\n\| Photo Credit:.*(?:\n.*)?')
    article = re.sub(pattern2, '', article)
    article = re.sub(r'\s+', ' ', article).strip() # removing extra white spaces
    df.at[index, 'processed_article'] = article


In [13]:
df.head()

,Title,Article,processed_article
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,Domestic passenger vehicle wholesales grew 26....
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...","Automobili Lamborghini has unveiled Urus S, th..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,International trade in the domestic currency w...
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,G-20 countries discussed the debt distress of ...
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India’s goods exports declined for the second ...


In [14]:
article = df['processed_article']

In [15]:
text = article[10]
print(text)

The Indian Railways turns 170 on April 16. Forever India’s favourite carrier, the railways are as much at a crossroads today as the rest of India — reconciling the trials and tribulations of the past with a future reckoning. Perhaps the most striking symbols of Indian Railways in the current zeitgeist are the talismanic Vande Bharat Express trains. Much of this modernisation contributes to passenger comfort, but sometimes it also leads to a spectacularly short-sighted loss of beauty and cultural heritage. In cities across India, colonial-era station buildings, many of which are a syncretic mix of local culture and European styles — such as Kacheguda in Hyderabad and Egmore in Chennai — are set for modernisation that will almost certainly produce drab, mall-like monoliths. Kacheguda station in Hyderabad is one among the 1,275 railway stations slated for redevelopment under the Amrit Bharat scheme. Egmore station in Chennai will be redeveloped at a cost of ₹734 crore. On April 8, the Pri

In [16]:
doc = nlp(text)
for sentence in doc.sents:
    print(sentence.text)

The Indian Railways turns 170 on April 16.
Forever India’s favourite carrier, the railways are as much at a crossroads today as the rest of India — reconciling the trials and tribulations of the past with a future reckoning.
Perhaps the most striking symbols of Indian Railways in the current zeitgeist are the talismanic Vande Bharat Express trains.
Much of this modernisation contributes to passenger comfort, but sometimes it also leads to a spectacularly short-sighted loss of beauty and cultural heritage.
In cities across India, colonial-era station buildings, many of which are a syncretic mix of local culture and European styles — such as Kacheguda in Hyderabad and Egmore in Chennai — are set for modernisation that will almost certainly produce drab, mall-like monoliths.
Kacheguda station in Hyderabad is one among the 1,275 railway stations slated for redevelopment under the Amrit Bharat scheme.
Egmore station in Chennai will be redeveloped at a cost of ₹734 crore.
On April 8, the Pri

In [17]:
df.shape

(2316, 3)

In [18]:


# Detect non-English articles
non_english = []
for index, row in df.iterrows():
    article = row['processed_article']
    lang = detect(article)
    if lang != 'en':
        non_english.append(index)

# Drop rows with non-English articles
df.drop(non_english, inplace=True)


In [19]:
non_english

[437, 439]

In [20]:
df.shape

(2314, 3)

In [21]:
df.head()

,Title,Article,processed_article
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,Domestic passenger vehicle wholesales grew 26....
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...","Automobili Lamborghini has unveiled Urus S, th..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,International trade in the domestic currency w...
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,G-20 countries discussed the debt distress of ...
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India’s goods exports declined for the second ...


In [22]:
df.to_csv('Articles.csv',index = False)

In [23]:
df.head()

,Title,Article,processed_article
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,Domestic passenger vehicle wholesales grew 26....
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...","Automobili Lamborghini has unveiled Urus S, th..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,International trade in the domestic currency w...
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,G-20 countries discussed the debt distress of ...
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India’s goods exports declined for the second ...


In [24]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.is_alpha and len(token)>2]
    return ' '.join(tokens)

In [25]:
df['processed_article'] = df['processed_article'].apply(preprocess_text)

In [26]:
df.head()

,Title,Article,processed_article
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,domestic passenger vehicle wholesale grow reco...
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...",Automobili Lamborghini unveil Urus successor o...
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,international trade domestic currency help red...
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,country discuss debt distress poor middle inco...
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India good export decline second successive mo...


In [27]:
df.dropna(subset=['processed_article'], inplace=True)

In [28]:
def extract_keywords(text):
    doc = nlp(text)
    keywords = []
    for token in doc:
        if not token.is_stop and token.is_alpha and len(token) > 2:
            keywords.append(token.lemma_)
    return list(set(keywords))

df['keywords'] = df['processed_article'].apply(extract_keywords)

In [29]:
df.head()

,Title,Article,processed_article,keywords
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,domestic passenger vehicle wholesale grow reco...,"[start, Aggarwal, surge, industry, efficient, ..."
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...",Automobili Lamborghini unveil Urus successor o...,"[start, Chairman, speed, sporty, endure, combi..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,international trade domestic currency help red...,"[Kolkata, industry, HDFC, Bank, body, July, tr..."
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,country discuss debt distress poor middle inco...,"[key, criticism, offer, distressed, Central, S..."
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India good export decline second successive mo...,"[gap, insignificant, Goods, surge, combine, pe..."


In [30]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [31]:
df.head()

,Title,Article,processed_article,keywords
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,domestic passenger vehicle wholesale grow reco...,"[start, Aggarwal, surge, industry, efficient, ..."
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...",Automobili Lamborghini unveil Urus successor o...,"[start, Chairman, speed, sporty, endure, combi..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,international trade domestic currency help red...,"[Kolkata, industry, HDFC, Bank, body, July, tr..."
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,country discuss debt distress poor middle inco...,"[key, criticism, offer, distressed, Central, S..."
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India good export decline second successive mo...,"[gap, insignificant, Goods, surge, combine, pe..."


In [32]:
type(df['processed_article'][0])

str

In [33]:
type(df['keywords'][0])

list

In [34]:
df['keywords'] = df['keywords'].apply(lambda x: ' '.join(x))

In [35]:
type(df['keywords'][0])

str

In [36]:
max_length = max(len(seq) for seq in df['processed_article'])
print(max_length)


14467


In [37]:
def tokenize_text(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )
    return encoding['input_ids'][0], encoding['attention_mask'][0]

In [38]:
df[['tokenized_article', 'attention_mask']] = df['processed_article'].apply(tokenize_text).apply(pd.Series)
df[['tokenized_keywords', 'attention_mask']] = df['keywords'].apply(tokenize_text).apply(pd.Series)


2023-05-15 17:11:01.515834: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 17:11:01.516029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 17:11:01.516065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 17:11:02.835235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-15 17:11:02.835319: I tensorflow/compile

In [39]:
df.head()

,Title,Article,processed_article,keywords,tokenized_article,attention_mask,tokenized_keywords
0,Passenger vehicles wholesales rise 26.73% to r...,Domestic passenger vehicle wholesales grew 26....,domestic passenger vehicle wholesale grow reco...,start Aggarwal surge industry efficient Direct...,"(tf.Tensor(101, shape=(), dtype=int32), tf.Ten...","(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso...","(tf.Tensor(101, shape=(), dtype=int32), tf.Ten..."
1,Lamborghini unveils Urus S at starting price o...,"Automobili Lamborghini has unveiled Urus S, th...",Automobili Lamborghini unveil Urus successor o...,start Chairman speed sporty endure combine app...,"(tf.Tensor(101, shape=(), dtype=int32), tf.Ten...","(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso...","(tf.Tensor(101, shape=(), dtype=int32), tf.Ten..."
2,Rupee trade arrangement to help cut transactio...,International trade in the domestic currency w...,international trade domestic currency help red...,Kolkata industry HDFC Bank body July trade tod...,"(tf.Tensor(101, shape=(), dtype=int32), tf.Ten...","(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso...","(tf.Tensor(101, shape=(), dtype=int32), tf.Ten..."
3,"G-20 Finance Ministers discuss debt distress, ...",G-20 countries discussed the debt distress of ...,country discuss debt distress poor middle inco...,key criticism offer distressed Central Shaktik...,"(tf.Tensor(101, shape=(), dtype=int32), tf.Ten...","(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso...","(tf.Tensor(101, shape=(), dtype=int32), tf.Ten..."
4,"Goods exports grew 6% , imports 16.5% this fiscal",India’s goods exports declined for the second ...,India good export decline second successive mo...,gap insignificant Goods surge combine peg Baro...,"(tf.Tensor(101, shape=(), dtype=int32), tf.Ten...","(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso...","(tf.Tensor(101, shape=(), dtype=int32), tf.Ten..."


In [40]:
# Convert titles to numerical labels
def convert_titles_to_labels(titles):
    label_map = {}
    labels = np.unique(titles)
    for idx, label in enumerate(labels):
        label_map[label] = idx
    label_ids = np.array([label_map[label] for label in titles], dtype=np.int32)
    return label_ids, label_map

In [41]:
title_labels, label_map = convert_titles_to_labels(df['Title'])

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenized_article_padded = pad_sequences(df['tokenized_article'], maxlen=max_length, padding='post')
attention_mask_padded = pad_sequences(df['attention_mask'], maxlen=max_length, padding='post')


In [44]:
# Calculate the split indices
train_size = int(len(tokenized_article_padded) * 0.8)
valid_size = int(len(tokenized_article_padded) * 0.1)

# Split the data and attention masks
train_data = tokenized_article_padded[:train_size]
train_attention_masks = attention_mask_padded[:train_size]
train_labels = title_labels[:train_size]

valid_data = tokenized_article_padded[train_size:train_size + valid_size]
valid_attention_masks = attention_mask_padded[train_size:train_size + valid_size]
valid_labels = title_labels[train_size:train_size + valid_size]

test_data = tokenized_article_padded[train_size + valid_size:]
test_attention_masks = attention_mask_padded[train_size + valid_size:]
test_labels = title_labels[train_size + valid_size:]


In [47]:
from transformers import TFBertModel
import tensorflow as tf

# Define the BERT model architecture
def build_model():
    input_ids = tf.keras.layers.Input(shape=(14467,), dtype=tf.int32)
    attention_mask = tf.keras.layers.Input(shape=(14467,), dtype=tf.int32)
    
    # Load the pre-trained BERT model
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    # Freeze the BERT model's layers to prevent training
    bert_model.trainable = False
    
    # Retrieve the BERT outputs
    bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
    pooled_output = bert_outputs.pooler_output
    
    # Add a fully connected layer for classification
    output = tf.keras.layers.Dense(len(label_map), activation='softmax')(pooled_output)
    
    # Construct the model
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
    
    return model

# Build the BERT model
model = build_model()


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [49]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [50]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit([train_data, train_attention_masks],
                    train_labels,
                    validation_data=([valid_data, valid_attention_masks], valid_labels),
                    epochs=5,
                    batch_size=8)


Epoch 1/5


2023-05-15 17:36:29.945907: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 74.85GiB (rounded to 80368930304)requested by op model_1/tf_bert_model_1/bert/encoder/layer_._0/attention/self/MatMul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-15 17:36:29.945996: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-05-15 17:36:29.946008: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 57, Chunks in use: 56. 14.2KiB allocated for chunks. 14.0KiB in use in bin. 348B client-requested in use in bin.
2023-05-15 17:36:29.946012: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 2, Chunks in use: 0. 1.5KiB allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-0

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/tf_bert_model_1/bert/encoder/layer_._0/attention/self/MatMul' defined at (most recent call last):
    File "/home/zoro/miniconda3/envs/nlp/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/zoro/miniconda3/envs/nlp/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/zoro/miniconda3/envs/nlp/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/zoro/miniconda3/envs/nlp/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/zoro/miniconda3/envs/nlp/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/zoro/miniconda3/envs/nlp/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      s